In [28]:
import csv

with open('Input/SGProduct.csv', 'r') as f: #['Id', 'InventoryCode', 'BarCode', 'HSNCode', 'Name', 'AliasName', 'DepartmentName', 'CategoryName', 'SubCategoryName', 'BrandName', 'UnitName', 'UnitCost' 11, 'AverageCost', 'Price' 13, 'VendorName', 'DepartmentCode', 'CategoryCode', 'SubCategoryCode', 'ProductType', 'ParentInventoryCode', 'AttributeName1', 'AttributeValue1', 'AttributeName2', 'AttributeValue2', 'AttributeName3', 'AttributeValue3', 'Publisher', 'Author', 'Origin', 'OriginPrice', 'VendorDiscountPerc', 'VendorDiscountAmount', 'Tax', 'TaxPercentage', 'Type', 'TradeIn', 'QtyOnHand', 'Remarks', 'MRP', 'Height', 'Width', 'Length', 'Weight', 'WeightUOM', 'OnlineSales', 'ImagePath', 'BinLocation', 'MinimumSellingPrice', 'SpecialPrice', 'ExportToWeighScale', 'POSWholesalePrice']
    reader = csv.reader(f)
    SGdata = [row for row in reader]
    
with open('Input/39Invoice.csv', 'r') as f: #['Barcode', 'Product Name']
    reader = csv.reader(f)
    invoice39data = [row for row in reader]

with open('Input/MYProduct.csv', 'r') as f: #['Id', 'InventoryCode', 'BarCode', 'HSNCode', 'Name', 'AliasName', 'DepartmentName', 'CategoryName', 'SubCategoryName', 'BrandName', 'UnitName', 'UnitCost' 11, 'AverageCost', 'Price' 13, 'VendorName', 'DepartmentCode', 'CategoryCode', 'SubCategoryCode', 'ProductType', 'ParentInventoryCode', 'AttributeName1', 'AttributeValue1', 'AttributeName2', 'AttributeValue2', 'Publisher', 'Author', 'Origin', 'OriginPrice', 'VendorDiscountPerc', 'VendorDiscountAmount', 'Tax', 'TaxPercentage', 'Type', 'TradeIn', 'QtyOnHand', 'Remarks', 'MRP', 'Height', 'Width', 'Length', 'Weight', 'WeightUOM', 'OnlineSales', 'ImagePath', 'BinLocation', 'MinimumSellingPrice', 'SpecialPrice', 'ExportToWeighScale']
    reader = csv.reader(f)
    MYdata = [row for row in reader]
    
with open('Input/ImportProductTemplate.csv', 'r') as f: #Template downloaded file
    reader = csv.reader(f)
    temp = [row for row in reader][0] #48 placeholders
    

In [29]:
def rateXchange_unit(n):
    n = float(n)
    n = n/3.53
    n = n + (n*0.09)
    n = n + 0.10
    return str(round(n,6))

def rateXchange_price(n):
    n = float(n)
    n = n/3.53
    n = n + (n*0.09)
    n = n + 0.10
    n = n // 0.01 * 0.01
    return str(round(n,1))

def removal(string):
    return string.replace(" ", "")

def listMaker(x):
    import ast
    x = ast.literal_eval(x)
    x = [n.strip() for n in x]
    return x


In [30]:
# Sorter

notInSGdata = [] #In Invoice and not in SG Inventory
data = [] #In Invoice and in SG Inventory with wrong prices
categoryNameSGNotNonedata = [] #In Invoice and in SG Inventory with correct prices



for i in invoice39data:
    nameInInvoice = i[1]
    
    length_data = len(data)
    length_categoryNameSGNotNonedata = len(categoryNameSGNotNonedata)
    
    for j in SGdata:
        nameInSGdata = j[4]
        categoryNameSG = j[7]
        
        if nameInInvoice == nameInSGdata and categoryNameSG == 'NONE':
            data.append(j)
        elif nameInInvoice == nameInSGdata and categoryNameSG !='NONE':
            categoryNameSGNotNonedata.append(j)
        else:
            pass
        
    if length_data == len(data) and length_categoryNameSGNotNonedata == len(categoryNameSGNotNonedata):
        notInSGdata.append(i)
        

In [134]:
# Prototype Purchase Invoice Maker

purchaseInvoiceList = [['Inventory Code', 'Name']]
for i in invoice39data:
    for j in data:
        
        if removal(i[1]) == removal(j[4]):
            purchaseInvoiceList.append([j[1], j[4]])
            
    for k in categoryNameSGNotNonedata:
        if removal(i[1]) == removal(k[4]):
            purchaseInvoiceList.append([k[1], k[4]])
            
    for x in notInSGdata:
        if removal(i[1]) == removal(x[1]):
            purchaseInvoiceList.append(['', x[1]])
        
        
with open('Input/test.csv','w',newline='') as f:
    writer=csv.writer(f)
    for row in purchaseInvoiceList:
        writer.writerow(row)


In [31]:
# Price Changer - Data alr in SG Inventory

for i in data:
    for j in MYdata:
        if removal(i[4]) == removal(j[4]): #sgName is the same as myName after removing all spaces
            unitCost = rateXchange_unit(j[11])
            price = rateXchange_price(j[13])
            i[11] = unitCost
            i[12] = '0'
            i[13] = price
            
print(len(data))


187


In [32]:
# Price Changer - Data in Invoice but not in SG Inventory

for i in notInSGdata:
    for j in MYdata:
        if removal(i[1]) == removal(j[4]): #invoiceName is the same as myName after removing all spaces
            unitCost = rateXchange_unit(j[11])
            price = rateXchange_price(j[13])
            
            i.append(unitCost)
            i.append(price)
            
notInSGdata

[['7622210816887', 'CADBURY ROAST ALMOND 90G', '1.804476', '2.7'],
 ['7622210400284', 'CADBURY DAIRY MILK BLACK FOREST 160G', '2.431303', '3.6'],
 ['8901233026497', 'C.DAIRY MILK LICKABLES 20G', '1.199263', '1.8'],
 ['9556182104049', 'CADBURY FRUIT & NUT 37G', '0.859603', '1.2'],
 ['9556182104162', 'CADBURY BLACK FOREST 37G', '0.859603', '1.2'],
 ['9556182104018', 'CADBURY ROAST ALMOND 37G', '0.859603', '1.2'],
 ['9556182104001', 'CADBURY DAIRY MILK 37G', '0.859603', '1.2'],
 ['9556182017264', 'DAIRY MILK CHOCLATE 15G', '0.374816', '0.5'],
 ['9556277401497', 'NONA KICAP LEMAK MANIS 325ML', '1.779773', '2.4'],
 ['9556277401497', 'NONA KICAP LEMAK MANIS 325ML', '1.779773', '2.4'],
 ['6901236387979', 'VINTA TISU KITCHEN TOWEL 80SHEET', '3.527479', '5.8']]

In [160]:
# Template Data Maker - Data alr in SG Inventory

dataForTemp = [['Id', 'InventoryCode', 'Barcode', 'Name', 'AliasName', 'DepartmentName', 'CategoryName', 'SubCategoryName', 'BrandName', 'UnitName', 'UnitCost', 'Price', 'VendorName', 'DepartmentCode', 'CategoryCode', 'SubCategoryCode', 'ProductType', 'ParentInventoryCode', 'AttributeName1', 'AttributeValue1', 'AttributeName2', 'AttributeValue2', 'AttributeName3', 'AttributeValue3', 'Publisher', 'Author', 'Origin', 'OriginPrice', 'VendorDiscountPerc', 'VendorDiscountAmount', 'Tax', 'Type', 'TradeIn', 'MRP', 'HSNCode', 'Length', 'Width', 'Height', 'Weight', 'Remarks', 'ImagePath', 'OnlineSales', 'WeightUOM', 'MinimumSellingPrice', 'SpecialPrice', 'POSWholesalePrice', 'BinLocation', 'ExportToWeighScale']]

for i in data:
    dataForTempList = []
    
    for j in range(48):
        dataForTempList.append('')
    
    dataForTempList[1] = i[1] # Inventory Code
    dataForTempList[2] = i[2] # Barcode
    dataForTempList[3] = i[4] # Name
    dataForTempList[5] = i[6] # Department Name
    dataForTempList[6] = i[7] # Category Name
    dataForTempList[9] = 'PCS' # Unit Name
    dataForTempList[10] = i[11] # Unit Cost
    dataForTempList[11] = i[13] # Price
    dataForTempList[12] = i[14] # Vendor Name
    dataForTempList[13] = i[15] # Department Code
    
    dataForTemp.append(dataForTempList)


In [33]:
# Template Data Maker - Data in Invoice but not in SG Inventory

dataForTemp = [['Id', 'InventoryCode', 'Barcode', 'Name', 'AliasName', 'DepartmentName', 'CategoryName', 'SubCategoryName', 'BrandName', 'UnitName', 'UnitCost', 'Price', 'VendorName', 'DepartmentCode', 'CategoryCode', 'SubCategoryCode', 'ProductType', 'ParentInventoryCode', 'AttributeName1', 'AttributeValue1', 'AttributeName2', 'AttributeValue2', 'AttributeName3', 'AttributeValue3', 'Publisher', 'Author', 'Origin', 'OriginPrice', 'VendorDiscountPerc', 'VendorDiscountAmount', 'Tax', 'Type', 'TradeIn', 'MRP', 'HSNCode', 'Length', 'Width', 'Height', 'Weight', 'Remarks', 'ImagePath', 'OnlineSales', 'WeightUOM', 'MinimumSellingPrice', 'SpecialPrice', 'POSWholesalePrice', 'BinLocation', 'ExportToWeighScale']]

for i in notInSGdata:
    dataForTempList = []
    
    for j in range(48):
        dataForTempList.append('')
    
    dataForTempList[2] = i[0] # Barcode
    dataForTempList[3] = i[1] # Name
    dataForTempList[5] = "MYR GOODS" # Department Name
    dataForTempList[6] = "NONE" # Category Name
    dataForTempList[9] = 'PCS' # Unit Name
    dataForTempList[10] = i[2] # Unit Cost
    dataForTempList[11] = i[3] # Price
    dataForTempList[12] = "AL KHWAJA MAJU ENTERPRISE" # Vendor Name
    
    dataForTemp.append(dataForTempList)

In [161]:
with open('Input/ImportProductTemplate_alrInSGInventory.csv','w',newline='') as f:
    writer=csv.writer(f)
    for row in dataForTemp:
        writer.writerow(row)

In [34]:
with open('Input/ImportProductTemplate_notInSGInventory.csv','w',newline='') as f:
    writer=csv.writer(f)
    for row in dataForTemp:
        writer.writerow(row)

In [ ]:
# In Invoice and In SGData with wrong prices
# In Invoice and In SGData with right prices
# In Invoice and not in SGData